# Open-Ended Capstone Data Exploration

In this notebook I will explore, clean, and transform multiple datasets. Each dataset is unique and must be handled in its own way. At the end of this exploration I will have gained familiarity with my datasets and their relationships.

In [1]:
# import libraries
import pandas as pd
import numpy as np

## Weather Data

The weather data was collected using an API which returned a JSON response. From this response I only collected fields which were relevant to my project. The data was relatively clean with the exception of the 'Location' column. In order to maintain as sense of uniformity across the NYC borough names I mapped the source data value 'Bronx County' to 'Bronx'.

In [2]:
df = pd.read_parquet('/Users/christopher/Desktop/Taxi_data_collection/weather_data_boroughs.parquet/')

In [3]:
df

,Time,Temp,Feels like,Main,Description,Visibility,Wind Speed,Location
0,2021-10-20 23:00:22,61.32,60.30,Clouds,scattered clouds,10000,5.01,Bronx County
1,2021-10-20 21:58:03,62.47,61.48,Clouds,broken clouds,10000,7.00,Bronx County
2,2021-10-20 22:50:02,61.39,60.33,Clear,clear sky,10000,3.44,Bronx County
3,2021-10-20 22:29:02,61.90,60.75,Clear,clear sky,10000,3.44,Bronx County
4,2021-10-20 22:01:28,62.22,61.20,Clouds,scattered clouds,10000,4.00,Bronx County
...,...,...,...,...,...,...,...,...
775,2021-10-20 23:04:03,60.24,59.16,Clear,clear sky,10000,1.01,Staten Island
776,2021-10-20 20:49:04,63.48,62.55,Clouds,scattered clouds,10000,3.44,Staten Island
777,2021-10-20 21:01:05,62.91,61.95,Clouds,scattered clouds,10000,0.00,Staten Island
778,2021-10-20 22:13:43,60.78,59.81,Clear,clear sky,10000,0.00,Staten Island


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Time         780 non-null    datetime64[ns]
 1   Temp         780 non-null    float64       
 2   Feels like   780 non-null    float64       
 3   Main         780 non-null    object        
 4   Description  780 non-null    object        
 5   Visibility   780 non-null    int64         
 6   Wind Speed   780 non-null    float64       
 7   Location     780 non-null    category      
dtypes: category(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 43.7+ KB


In [5]:
#uniformity for bronx county = bronx
bronx_dict = {'Bronx County':'Bronx'}
df['Location'] = df['Location'].apply(lambda x: bronx_dict[x] if x in bronx_dict.keys() else x)

In [6]:
df.head()

,Time,Temp,Feels like,Main,Description,Visibility,Wind Speed,Location
0,2021-10-20 23:00:22,61.32,60.30,Clouds,scattered clouds,10000,5.01,Bronx
1,2021-10-20 21:58:03,62.47,61.48,Clouds,broken clouds,10000,7.00,Bronx
2,2021-10-20 22:50:02,61.39,60.33,Clear,clear sky,10000,3.44,Bronx
3,2021-10-20 22:29:02,61.90,60.75,Clear,clear sky,10000,3.44,Bronx
4,2021-10-20 22:01:28,62.22,61.20,Clouds,scattered clouds,10000,4.00,Bronx


In [7]:
df['Location'].value_counts()

Staten Island    156
Queens           156
Manhattan        156
Brooklyn         156
Bronx            156
Name: Location, dtype: int64

## Traffic Sensor Data

The traffic sensor data is collected from 'http://207.251.86.229/nyc-links-cams/LinkSpeedQuery.txt' and is updated real-time. The dataset contained a lot of redundant and unecessary columns which were removed. The data also contained sensor data with a time stamp of '1978-01-01'. According to the data FAQ's broken traffic sensors are given an old data and can be discarded.

In [8]:
df = pd.read_parquet('/Users/christopher/Desktop/Taxi_data_collection/traffic_sensor_feed.parquet/')

In [9]:
df.head()

,id,Speed,TravelTime,Status,DataAsOf,linkId,linkPoints,EncodedPolyLine,EncodedPolyLineLvls,Owner,Transcom_id,linkName,Borough
0,126,57.16,130,0,11/17/2021 00:50:09,4616247,"40.8271606,-73.84993 40.82771,-73.84671 40.828...",w`exF~xvaMmBaSkCiSw@iLScPW}Hy@eH_BmHcDeIaIaO{D...,BBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616247,BE N Castle Hill Avenue - Griswold Ave,Bronx
1,129,57.16,81,0,11/17/2021 00:50:09,4616246,"40.8240706,-73.874311 40.8247,-73.86959 40.825...",mmdxFjq{aM}Bm\aBwUa@sIeB_\qCq`@sB{Y,BBBBBBB,NYC_DOT_LIC,4616246,BE N STRATFORD AVENUE - CASTLE HILL AVE,Bronx
2,137,41.01,109,0,11/17/2021 00:50:09,4616260,"40.8242005,-73.874361 40.8249804,-73.868411 40...",gndxFvq{aM{Ced@sAqReBa^{Cyb@{Bw[,BBBBBB,NYC_DOT_LIC,4616260,BE S CASTLE HILL AVENUE - STRATFORD AVENUE,Bronx
3,142,41.01,183,0,11/17/2021 00:50:09,4616261,"40.83037,-73.85062 40.82996,-73.849251 40.8294...",ytexFj}vaMpAqGbBgHrAgGl@sF?oEa@cGoDu\e@cGAsEF}...,BBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616261,BE S Griswold - Castle Hill Avenue,Bronx
4,159,42.25,130,0,11/17/2021 00:50:09,4616252,"40.8563506,-73.87233 40.85219,-73.871371 40.85...",ewjxF`e{aM~X_EfLs@pRFbE^fUlClPlC`TdE`Gb@|HMtG]...,BBBBBBBBBBBBB,NYC_DOT_LIC,4616252,BRP N WATSON AVENUE - FORDHAM ROAD,Bronx


In [10]:
df.Borough.value_counts()

Queens           492
Staten Island    420
Manhattan        420
Bronx            288
Brooklyn         216
Name: Borough, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   id                   1836 non-null   int64   
 1   Speed                1836 non-null   float64 
 2   TravelTime           1836 non-null   int64   
 3   Status               1836 non-null   int64   
 4   DataAsOf             1836 non-null   object  
 5   linkId               1836 non-null   int64   
 6   linkPoints           1836 non-null   object  
 7   EncodedPolyLine      1836 non-null   object  
 8   EncodedPolyLineLvls  1836 non-null   object  
 9   Owner                1836 non-null   object  
 10  Transcom_id          1836 non-null   int64   
 11  linkName             1836 non-null   object  
 12  Borough              1836 non-null   category
dtypes: category(1), float64(1), int64(5), object(6)
memory usage: 174.2+ KB


In [12]:
df['DataAsOf'] = pd.to_datetime(df['DataAsOf'])

In [13]:
# remove columns which are not needed
df.drop(columns=['Status','linkId','Transcom_id','Owner','EncodedPolyLine','EncodedPolyLineLvls'],inplace=True)

In [14]:
df['DataAsOf'].min()

Timestamp('1978-01-01 00:00:00')

In [15]:
#filter out broken sensors data
from datetime import date,timedelta
# print(date.today() - timedelta(1))
date_filter = date.today() - timedelta(1) #1 day buffer
date_filter = pd.to_datetime(date_filter)
df = df[df['DataAsOf'] > date_filter]

In [16]:
df.head()

,id,Speed,TravelTime,DataAsOf,linkPoints,linkName,Borough
0,126,57.16,130,2021-11-17 00:50:09,"40.8271606,-73.84993 40.82771,-73.84671 40.828...",BE N Castle Hill Avenue - Griswold Ave,Bronx
1,129,57.16,81,2021-11-17 00:50:09,"40.8240706,-73.874311 40.8247,-73.86959 40.825...",BE N STRATFORD AVENUE - CASTLE HILL AVE,Bronx
2,137,41.01,109,2021-11-17 00:50:09,"40.8242005,-73.874361 40.8249804,-73.868411 40...",BE S CASTLE HILL AVENUE - STRATFORD AVENUE,Bronx
3,142,41.01,183,2021-11-17 00:50:09,"40.83037,-73.85062 40.82996,-73.849251 40.8294...",BE S Griswold - Castle Hill Avenue,Bronx
4,159,42.25,130,2021-11-17 00:50:09,"40.8563506,-73.87233 40.85219,-73.871371 40.85...",BRP N WATSON AVENUE - FORDHAM ROAD,Bronx


# Taxi Data

The taxi datasets contained data on every NYC yellow taxi ride from 2015-2021. There were multiple issues that needed to be addressed when deciding on how to transform and store these datasets. One major issue was that a couple of the columns contained unique numerical identifiers/values which mapped to its actual value (ex. locationID 2 = Borough Queens). In order to provide a rich and insightful dataset I downloaded multiple mapping documents from the sites data dictionary. This helped me include actual values for a lot of these columns.

Another issue involved data from 2015-2017 which included Lat/Lon columns. Datasets after 2018 included a unique identifier which could be used to map back to it's original borough value, similar to what I mentioned earlier. In order to correct this issue I reverse geo-coded the Lat/Lon columns to their zip code equivalents using a Python package called 'uszipcode'. I then collected and merged two zip code datasets into a dictionary, after which I could use to map NYC zip codes to their borough. 

In [17]:
#load LocationID to Borough mapping file
location_data_map = pd.read_csv('/Users/christopher/Downloads/taxi+_zone_lookup.csv')

In [18]:
location_data_map

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [19]:
location_dic = dict(zip(location_data_map['LocationID'],location_data_map['Borough']))

In [20]:
location_dic

{1: 'EWR',
 2: 'Queens',
 3: 'Bronx',
 4: 'Manhattan',
 5: 'Staten Island',
 6: 'Staten Island',
 7: 'Queens',
 8: 'Queens',
 9: 'Queens',
 10: 'Queens',
 11: 'Brooklyn',
 12: 'Manhattan',
 13: 'Manhattan',
 14: 'Brooklyn',
 15: 'Queens',
 16: 'Queens',
 17: 'Brooklyn',
 18: 'Bronx',
 19: 'Queens',
 20: 'Bronx',
 21: 'Brooklyn',
 22: 'Brooklyn',
 23: 'Staten Island',
 24: 'Manhattan',
 25: 'Brooklyn',
 26: 'Brooklyn',
 27: 'Queens',
 28: 'Queens',
 29: 'Brooklyn',
 30: 'Queens',
 31: 'Bronx',
 32: 'Bronx',
 33: 'Brooklyn',
 34: 'Brooklyn',
 35: 'Brooklyn',
 36: 'Brooklyn',
 37: 'Brooklyn',
 38: 'Queens',
 39: 'Brooklyn',
 40: 'Brooklyn',
 41: 'Manhattan',
 42: 'Manhattan',
 43: 'Manhattan',
 44: 'Staten Island',
 45: 'Manhattan',
 46: 'Bronx',
 47: 'Bronx',
 48: 'Manhattan',
 49: 'Brooklyn',
 50: 'Manhattan',
 51: 'Bronx',
 52: 'Brooklyn',
 53: 'Queens',
 54: 'Brooklyn',
 55: 'Brooklyn',
 56: 'Queens',
 57: 'Queens',
 58: 'Bronx',
 59: 'Bronx',
 60: 'Bronx',
 61: 'Brooklyn',
 62: 'Broo

In [21]:
# load newer dataset with LocationID columns
df = pd.read_csv('/Users/christopher/Desktop/Taxi_data_collection/taxi_data_2018.csv',nrows=2200)

In [22]:
# map LocationID columns to borough value
df['PULocationID'] = df['PULocationID'].apply(lambda x: location_dic[x])
df['DOLocationID'] = df['DOLocationID'].apply(lambda x: location_dic[x])

In [23]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-12-01 00:28:22,2018-12-01 00:44:07,2,2.5,1,N,Manhattan,Manhattan,1,12.0,0.5,0.5,3.95,0.0,0.3,17.25
1,1,2018-12-01 00:52:29,2018-12-01 01:11:37,3,2.3,1,N,Manhattan,Manhattan,1,13.0,0.5,0.5,2.85,0.0,0.3,17.15
2,2,2018-12-01 00:12:52,2018-12-01 00:36:23,1,0.0,1,N,Manhattan,Queens,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
3,1,2018-12-01 00:35:08,2018-12-01 00:43:11,1,3.9,1,N,Queens,Queens,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55
4,1,2018-12-01 00:21:54,2018-12-01 01:15:13,1,12.8,1,N,Manhattan,Brooklyn,1,45.0,0.5,0.5,9.25,0.0,0.3,55.55


In [24]:
# load older dataset with Lat/Lon columns
df = pd.read_csv('/Users/christopher/Desktop/Taxi_data_collection/taxi_data_2016.csv',nrows=1000)

In [25]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               1000 non-null   int64  
 1   tpep_pickup_datetime   1000 non-null   object 
 2   tpep_dropoff_datetime  1000 non-null   object 
 3   passenger_count        1000 non-null   int64  
 4   trip_distance          1000 non-null   float64
 5   pickup_longitude       1000 non-null   float64
 6   pickup_latitude        1000 non-null   float64
 7   RatecodeID             1000 non-null   int64  
 8   store_and_fwd_flag     1000 non-null   object 
 9   dropoff_longitude      1000 non-null   float64
 10  dropoff_latitude       1000 non-null   float64
 11  payment_type           1000 non-null   int64  
 12  fare_amount            1000 non-null   float64
 13  extra                  1000 non-null   float64
 14  mta_tax                1000 non-null   float64
 15  tip_a

In [27]:
#convert datatypes to float
df['pickup_longitude'] = df['pickup_longitude'].astype(float)
df['pickup_latitude'] = df['pickup_latitude'].astype(float)
df['dropoff_longitude'] = df['dropoff_longitude'].astype(float)
df['dropoff_latitude'] = df['dropoff_latitude'].astype(float)

In [28]:
#import uszipcode and convert lat, lon columns to their zip code equivalents
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode
import numpy as np

def get_zipcode(lat, lon):
    """ This function intakes a set of (lat,long) coordinates and returns the zipcode location."""
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    try:
        result = result[0].zipcode
    except:
        result = 0
    return result


df['pu_zipcode'] = df.apply(lambda x: get_zipcode(x.pickup_latitude,x.pickup_longitude),axis=1)
df['do_zipcode'] = df.apply(lambda x: get_zipcode(x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [29]:
#load zip code mapping #1
zip_data = pd.read_csv('/Users/christopher/Downloads/csvData.csv')

zip_data = zip_data[zip_data['county'].isin(['Queens','Kings','Bronx','New York','Richmond'])]

zip_data['county'] = zip_data['county'].map({'Queens':'Queens','Kings':'Brooklyn','Bronx':'Bronx',
                                            'New York':'Manhattan','Richmond':'Staten Island'})

zipcode_dic = dict(zip(zip_data['zip'],zip_data['county']))
zipcode_dic[0] = 'Unknown'

#load zip code mapping #2
zip_data_2 = pd.read_csv('/Users/christopher/Downloads/nyc_zip_borough_neighborhoods_pop.csv')

zipcode_dic_2 = dict(zip(zip_data_2['zip'],zip_data_2['borough']))

zipcode_dic.update(zipcode_dic_2)

#master dictionary with zip code to borough key:value pairs
zipcode_dic

{11368: 'Queens',
 11385: 'Queens',
 11211: 'Brooklyn',
 11208: 'Brooklyn',
 10467: 'Bronx',
 11236: 'Brooklyn',
 11226: 'Brooklyn',
 11373: 'Queens',
 11234: 'Brooklyn',
 11220: 'Brooklyn',
 10456: 'Bronx',
 11214: 'Brooklyn',
 10025: 'Manhattan',
 11207: 'Brooklyn',
 10314: 'Staten Island',
 11219: 'Brooklyn',
 11206: 'Brooklyn',
 11230: 'Brooklyn',
 10458: 'Bronx',
 11377: 'Queens',
 11221: 'Brooklyn',
 11229: 'Brooklyn',
 10453: 'Bronx',
 11223: 'Brooklyn',
 11355: 'Queens',
 11233: 'Brooklyn',
 10468: 'Bronx',
 11235: 'Brooklyn',
 10029: 'Manhattan',
 11204: 'Brooklyn',
 11203: 'Brooklyn',
 10462: 'Bronx',
 11212: 'Brooklyn',
 10452: 'Bronx',
 10466: 'Bronx',
 10457: 'Bronx',
 10002: 'Manhattan',
 11375: 'Queens',
 11218: 'Brooklyn',
 10469: 'Bronx',
 10463: 'Bronx',
 11215: 'Brooklyn',
 11691: 'Queens',
 10472: 'Bronx',
 11209: 'Brooklyn',
 11213: 'Brooklyn',
 10027: 'Manhattan',
 11210: 'Brooklyn',
 11434: 'Queens',
 11432: 'Queens',
 11201: 'Brooklyn',
 10032: 'Manhattan',
 113

In [30]:
#map zipcodes to borough
df['PULocationID'] = df['pu_zipcode'].apply(lambda x: zipcode_dic[int(x)] if int(x) in list(zipcode_dic.keys()) else 'Unknown')
df['DOLocationID'] = df['do_zipcode'].apply(lambda x: zipcode_dic[int(x)] if int(x) in list(zipcode_dic.keys()) else 'Unknown')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               1000 non-null   int64  
 1   tpep_pickup_datetime   1000 non-null   object 
 2   tpep_dropoff_datetime  1000 non-null   object 
 3   passenger_count        1000 non-null   int64  
 4   trip_distance          1000 non-null   float64
 5   pickup_longitude       1000 non-null   float64
 6   pickup_latitude        1000 non-null   float64
 7   RatecodeID             1000 non-null   int64  
 8   store_and_fwd_flag     1000 non-null   object 
 9   dropoff_longitude      1000 non-null   float64
 10  dropoff_latitude       1000 non-null   float64
 11  payment_type           1000 non-null   int64  
 12  fare_amount            1000 non-null   float64
 13  extra                  1000 non-null   float64
 14  mta_tax                1000 non-null   float64
 15  tip_a

In [34]:
#collected from source data dictionary, mapping payment types
payment_dict = {1:'Credit card', 2:'Cash', 3:'No charge', 4:'Dispute', 5:'Unknown', 6:'Voided trip'}
df['payment_type'] = df['payment_type'].apply(lambda x: payment_dict[x])

In [35]:
#collected from source data dictionary, mapping rate types
rate_dic = {1:'Standard rate', 2:'JFK', 3:'Newark', 4:'Nassau or Westchester', 5:'Negotiated fare', 6:'Group ride'}
df['RatecodeID'] = df['RatecodeID'].apply(lambda x: rate_dic[x])

In [36]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pu_zipcode,do_zipcode,PULocationID,DOLocationID
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,Standard rate,N,-73.981842,...,0.5,0.5,0.0,0.0,0.3,8.8,10003,10009,Manhattan,Manhattan
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,Standard rate,N,-73.944473,...,0.5,0.5,0.0,0.0,0.3,19.3,10009,11211,Manhattan,Brooklyn
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,Standard rate,N,-73.950272,...,0.5,0.5,0.0,0.0,0.3,34.3,11217,10029,Brooklyn,Manhattan
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,Standard rate,N,-73.962242,...,0.0,0.5,0.0,0.0,0.3,17.3,10002,11225,Manhattan,Brooklyn
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,Standard rate,N,-73.977264,...,0.0,0.5,0.0,0.0,0.3,8.8,10028,10111,Manhattan,Manhattan


## Vehicle Collisions Data

This dataset contained all vehicle collisions within NYC which amounted to more than $1000 of damage. The data was relatively clean and contained a lot of interesting features. In terms of transformations I decided to combine both the crash date and crash time columns. I also created a rush hour flag column which denoted whether an accident happened during rush hour. This column will be helpful to gain further insights within this dataset.

In [37]:
df = pd.read_csv('/Users/christopher/Desktop/Taxi_data_collection/vehicle_collisions.csv', nrows=1000)

In [38]:
df.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street_name', 'off_street_name',
       'cross_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5'],
      dtype='object')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   crash_date                     1000 non-null   object 
 1   crash_time                     1000 non-null   object 
 2   borough                        633 non-null    object 
 3   zip_code                       633 non-null    float64
 4   latitude                       911 non-null    float64
 5   longitude                      911 non-null    float64
 6   location                       911 non-null    object 
 7   on_street_name                 755 non-null    object 
 8   off_street_name                466 non-null    object 
 9   cross_street_name              245 non-null    object 
 10  number_of_persons_injured      1000 non-null   int64  
 11  number_of_persons_killed       1000 non-null   int64  
 12  number_of_pedestrians_injured  1000 non-null   in

In [40]:
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-04-14T00:00:00.000,5:32,NaN,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4407480,Sedan,Sedan,NaN,NaN,NaN
1,2021-04-13T00:00:00.000,21:35,BROOKLYN,11217.0,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,620 ATLANTIC AVENUE,...,NaN,NaN,NaN,NaN,4407147,Sedan,NaN,NaN,NaN,NaN
2,2021-04-15T00:00:00.000,16:15,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,...,NaN,NaN,NaN,NaN,4407665,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,2021-04-13T00:00:00.000,16:00,BROOKLYN,11222.0,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,...,Unspecified,NaN,NaN,NaN,4407811,Sedan,NaN,NaN,NaN,NaN
4,2021-04-12T00:00:00.000,8:25,NaN,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4406885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [41]:
df.borough.value_counts()

BROOKLYN         253
QUEENS           159
BRONX            107
MANHATTAN         90
STATEN ISLAND     24
Name: borough, dtype: int64

In [42]:
# titlecase borough names
df.borough = df.borough.str.title()

In [43]:
# combine crash date + crash time columns
df['crash_date'] = pd.to_datetime(df['crash_date'])
df['crash_date'] = df['crash_date'].dt.strftime('%m-%d-%Y')
df['crash_date_time'] = df['crash_date'] + ' ' + df['crash_time']
df['crash_date_time'] = pd.to_datetime(df['crash_date_time'])

In [44]:
#create rush hour flag columns,rush hour 8-9am or 3-7pm
df.loc[((df['crash_date_time'].dt.hour >= 8) & (df['crash_date_time'].dt.hour < 10)) | ((df['crash_date_time'].dt.hour >= 15) & (df['crash_date_time'].dt.hour < 20)), 'rush_hour'] = 1

In [45]:
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,crash_date_time,rush_hour
0,04-14-2021,5:32,NaN,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,...,NaN,NaN,4407480,Sedan,Sedan,NaN,NaN,NaN,2021-04-14 05:32:00,NaN
1,04-13-2021,21:35,Brooklyn,11217.0,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,620 ATLANTIC AVENUE,...,NaN,NaN,4407147,Sedan,NaN,NaN,NaN,NaN,2021-04-13 21:35:00,NaN
2,04-15-2021,16:15,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,...,NaN,NaN,4407665,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2021-04-15 16:15:00,1.0
3,04-13-2021,16:00,Brooklyn,11222.0,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,...,NaN,NaN,4407811,Sedan,NaN,NaN,NaN,NaN,2021-04-13 16:00:00,1.0
4,04-12-2021,8:25,NaN,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,NaN,...,NaN,NaN,4406885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,2021-04-12 08:25:00,1.0
